In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
import pickle
from itertools import product

import numpy as np
import pandas as pd
import scipy.stats as stats

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns

sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')

In [ ]:
repo_dir = Path('../..')

In [ ]:
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))
    
from analysis.curve_fitting.src.fitting_functions import LOSS_FUNCTIONS
from analysis.curve_fitting.src.utils import apply_filters, load_yaml, convert_loss_parameters, convert_loss_parameters_batch

from visualization.src.utils import COLOR_PALETTES, set_ticks, save_figs
from visualization.src.visualize import plot_reg, plot_reg_bivariate, plot_confidence_intervals



In [ ]:
args = {
    'results_csv': repo_dir / 'results' / 'benchmark_scores.csv',
}

In [ ]:
results_csv = args['results_csv']

df_results = pd.read_csv(results_csv)

## Load Experiment Configuration

In [ ]:
config = {
    'data_filters': {
        'set_filters': {
            'region': [
                'V1',
                'V2',
                'V4',
                'IT',
                'Behavioral'
                ],
            'dataset': [
                'imagenet',
                'ecoset',
                ]
            },
    'boolean_filters': {
        'equals_false': [
            'is_pretrained',
            'is_random',
            'is_ssl',
            'is_adv',
            'is_ablation'
            ]
        },
    
    # 'group_by': {
    #     'avg_score': {
    #         'keys': [
    #             'model_id',
    #             'arch',
    #             'dataset',
    #             'flops',
    #             'n_params',
    #             'n_samples',
    #             'n_samples_seen',
    #             'total_flops',
    #             'arch_family',
    #             'samples_per_class',
    #             'benchmark_name',
    #             'region'
    #         ],
    #         'reduce': {'score': 'mean'}}},

    'combine_arch_families': True,
    }
}

## Apply Data Filters

In [ ]:
df = apply_filters(df_results, config.get('data_filters', {}))
df['benchmark_type'] = 'public'

## Load Private Benchmark Results

In [ ]:
private_results = repo_dir / 'results' / 'benchmark_scores_brainscore.csv'
df_private = pd.read_csv(private_results)

In [ ]:

model_ids = df.model_id.unique()
df_private = df_private[df_private.model_id.isin(model_ids)]
columns = [
    'model_id',
    'FreemanZiemba2013.V1-pls',
    'FreemanZiemba2013.V2-pls',
    'MajajHong2015.V4-pls',
    'MajajHong2015.IT-pls',
    'Rajalingham2018-i2n',
]
df_private = df_private[columns]

column_map = {
    'FreemanZiemba2013.V1-pls': 'movshon.FreemanZiemba2013public.V1-pls',
    'FreemanZiemba2013.V2-pls': 'movshon.FreemanZiemba2013public.V2-pls',
    'MajajHong2015.V4-pls': 'dicarlo.MajajHong2015public.V4-pls',
    'MajajHong2015.IT-pls': 'dicarlo.MajajHong2015public.IT-pls',
    'Rajalingham2018-i2n': 'dicarlo.Rajalingham2018public-i2n',
}
df_private = df_private.rename(columns=column_map)
df_private = df_private.melt(id_vars='model_id', value_vars=list(column_map.values())[:], var_name='benchmark_name', value_name='score')


region_map = {
    'movshon.FreemanZiemba2013public.V1-pls': 'V1',
    'movshon.FreemanZiemba2013public.V2-pls': 'V2',
    'dicarlo.MajajHong2015public.V4-pls': 'V4',
    'dicarlo.MajajHong2015public.IT-pls': 'IT',
    'dicarlo.Rajalingham2018public-i2n': 'Behavioral',
}
# df_private['benchmark_type'] = 'private'
df_private['region'] = df_private.benchmark_name.apply(lambda x: region_map[x])
df_private

In [ ]:
df_private.region.unique()

In [ ]:
df_public = df.copy().drop(columns=['benchmark_type'])
df_plot = df_public.merge(df_private, on=['model_id', 'benchmark_name', 'region'], how='inner', suffixes=('_public', '_private'))
df_plot.columns

### Compute confidence intervals

In [ ]:
random_state = 42
num_bootstraps = 1000
fraction = 1

rng = np.random.default_rng(random_state)
all_samples = int(len(df_plot)/5)
num_samples = int(fraction * all_samples)
random_indices = rng.integers(0, all_samples, size=(num_bootstraps, num_samples))

In [ ]:
df_fits = []

for i, region in tqdm(enumerate(['V1', 'V2', 'V4', 'IT', 'Behavioral']), total=5):
    for j, indices in tqdm(enumerate(random_indices), leave=False, total=num_bootstraps):
        data = df_plot[df_plot.region == region].copy().reset_index(drop=True)
        data = data.iloc[indices]
        data = data.dropna(subset=['score_public', 'score_private'])
        x = data['score_public']
        y = data['score_private']
        reg = stats.linregress(x, y)
        
        df_fits.append({
            'region': region,
            'slope': reg.slope,
            'intercept': reg.intercept,
            'rvalue': reg.rvalue,
            'pvalue': reg.pvalue,
            'stderr': reg.stderr,
        })
        
df_fits = pd.DataFrame(df_fits)
df_fits

## Visualize

#### Plotting settings

In [ ]:
linewidth = 3.0
scatter_alpha = 0.2
fig_multiplier = 1
font_size_multiplier = 1.5
figsize = (12, 6)
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])

color_palaette_samples = COLOR_PALETTES['samples']
color_palaette_regions = COLOR_PALETTES['regions']
color = color_palaette_regions[-1]

#### Plot

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=figsize, dpi=300)
for i, region in enumerate(['V1', 'V2', 'V4', 'IT', 'Behavioral']):
    ax = axes.flatten()[i]
    data = df_plot[df_plot.region == region]
    data = data.dropna(subset=['score_public', 'score_private'])
    x = data['score_public']
    y = data['score_private']
    reg = stats.linregress(x, y)
    
    ### Plot
    text = f"$R^2$: {reg.rvalue**2:.2f}\np-value ={reg.pvalue:.2e}"
    sns.regplot(x=x, y=y, ax=ax, color=color, label=text)
    
    ### Formatting
    ax.set_title(region, fontsize=16, fontweight='bold')
    ax.set_xlabel('Public', fontsize=12, fontweight='bold')
    ax.set_ylabel('Private', fontsize=12, fontweight='bold')
    ax.grid(True)
    
    ### Legend
    ax.legend()
    ax.spines[['right', 'top']].set_visible(False)


ax = axes.flatten()[-1]
data = df_plot.groupby(['model_id']).mean({'score_private':'mean', 'score_public':'mean'}).reset_index()
data = data.dropna(subset=['score_public', 'score_private'])
x = data['score_public']
y = data['score_private']
reg = stats.linregress(x, y)

### Plot
text = f"$R^2$: {reg.rvalue**2:.2f}\np-value ={reg.pvalue:.2e}"
sns.regplot(x=x, y=y, ax=ax, color=color, label=text)

### Formatting
ax.set_title('Average', fontsize=16, fontweight='bold')
ax.set_xlabel('Public', fontsize=12, fontweight='bold')
ax.set_ylabel('Private', fontsize=12, fontweight='bold')
ax.grid(True)

### Legend
ax.legend()
ax.spines[['right', 'top']].set_visible(False)

plt.tight_layout()


figures_dir = '../figures'
fig_name = 'fig8_public_vs_private'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)